# Imports and functions

In [1]:
import torch
import pickle
from transformers import BertModel, BertTokenizerFast
from tqdm import tqdm_notebook as tqdm

In [2]:
def save_object(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)


def load_object(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

def read_raw_questions(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    lines = list(filter(lambda line: ('באמצעות קישור ההזמנה' not in line), lines))
    lines = list(filter(None, lines))
    lines = list(filter(lambda line: ('<המדיה לא נכללה>' not in line), lines))
    lines = list(filter(lambda line: ('הודעה זו נמחקה' not in line), lines))
    lines = [':'.join(line.split(':')[2:])[:512] for line in lines] #remove time and id
    lines = list(filter(None, lines))
    lines = list(filter(lambda line: (len(line.split(' '))>4), lines)) #remove short messeges
    return lines

# Read and clean chat history

In [3]:
lines = read_raw_questions('history.txt')

In [4]:
lines

[' שלום הרב, אדם המקבל שבת בפלג המנחה, האם הוא יכול לעשות קידוש וסעודת שבת לפני צאת הכוכבים?',
 ' שלום הרב, האם יש אפשרות לחמם רוטב בבוקר?על פלטה שדולקת מערב שבת ברצף?',
 ' שלום לרב, היכן על אדם המתארח אצל השכנים בסעודת שבת וחוזר ללון בביתו להדליק נרות?',
 ' כן. וישתדל לאכול כזית אחרי צאת הכוכבים.',
 ' תודה רבה הרב, שבת שלום.',
 ' תודה רבה, שבת שלום!',
 ' הרב, האם חייבת להיות דלוקה ברצף?',
 ' הפוך, אם זה בשעון שבת, מותר גם לספרדים. ( אם מניחים לפני שנדלק)',
 ' בסדר גמור, תודה רבה הרב ושבת שלום!!',
 ' חמץ שנשאר בחדרים של הישיבה , האם אפשר לסמוך על מכירת החמץ של רב העיר?',
 ' שלום הרב בישלו אוכל חלבי בסיר חלבי אך המכסה משמש לבשר (המכסה הוא זכוכית והשוליים ברזל) מעריך שלא בן יומו, והאוכל נגע במכסה מה הדין האוכל /סיר /מכסה?',
 ' הרב משקעות מוגזים מחול ללא כשרות.',
 ' על המכירה של הישיבה, כן',
 ' שאלה עובדתית. ויכוח  על עובדות. איך אביע דעה בזה?',
 ' האוכל והסיר מותרים. המכסה צריך הגעלה.',
 ' המקל תבוא עליו הברכה ?',
 ' צריך להגיד משהו בשילוח הקן?',
 ' היונה הלכה לקחתי את הביצה שלה פשוט',
 

# Import hebrew model

In [5]:
alephbert_tokenizer = BertTokenizerFast.from_pretrained('onlplab/alephbert-base')
alephbert = BertModel.from_pretrained('onlplab/alephbert-base')

Some weights of the model checkpoint at onlplab/alephbert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
alephbert.pooler.activation = torch.nn.Identity()
alephbert.eval()
if torch.cuda.is_available():
    alephbert.cuda()

# Create embedding space

In [12]:
batch_size = 10
embedding_space = []
for i in tqdm(range((len(lines)//batch_size) + 1)):
    tokens = alephbert_tokenizer(lines[i*batch_size:(i+1)*batch_size], padding=True, return_tensors="pt")
    with torch.no_grad():
        embedding_batch = alephbert(**tokens)['pooler_output']
    embedding_space.append(embedding_batch)
    save_object(embedding_space, 'embedding_space_checkpoint.pkl')
    
embedding_space = torch.cat(embedding_space)
save_object(embedding_space, 'embedding_space.pkl')

with open('embedding_space_questions.txt', 'w') as f:
    for line in lines:
        f.write("%s\n" % line)

/tmp/ipykernel_774/1497132272.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range((len(lines)//batch_size) + 1)):


  0%|          | 0/1755 [00:00<?, ?it/s]

# Inference

In [8]:
embedding_space = load_object('embedding_space.pkl')
with open('embedding_space_questions.txt') as f:
        lines = f.read().splitlines()

In [9]:
embedding_space.shape, len(lines)

(torch.Size([17540, 768]), 17546)

In [10]:
new_questrion = ["שלום הרב. אם הזמנתי צ'יפס בנפרד ממסעדה בשרית, ואכלתי אותו, האם צריך להמתין 6 שעות? תודה רהה"]

In [11]:
tokens = alephbert_tokenizer(new_questrion, padding=True, return_tensors="pt")

with torch.no_grad():
    new_embedding = alephbert(**tokens)['pooler_output']

dist = torch.norm(embedding_space - new_embedding, dim=1, p=None)

knn = dist.topk(5, largest=False)

for i,d in zip(knn.indices,knn.values):
    print(lines[i], 'score: %0.2f'%d)

 שלום הרב. אם הזמנתי צ'יפס בנפרד ממסעדה בשרית, ואכלתי אותו, האם צריך להמתין 6 שעות? תודה רהה score: 0.00
 שלום הרב, הזמנתי משלוח ממסעדה בשרית והגיעה לאפה ריקה בשקית נייר בנפרד מהמנה הבשרית, האם מותר לאכול אותה עם גבינה? score: 5.05
 שלום הרב, ניתן לבלוע כדור פרוביוטיקה חלבי לאחר ארוחה בשרית? ואם לא האם צריך להמתין שש שעות? תודה רבה score: 5.85
 שלום הרב, צריך תזכורת- אם בישלתי/טיגנתי אוכל במחבת בשרית בת יומה- מותר לאכול את האוכל עם חלב? score: 5.89
 שלום הרב, האורז של הישיבה אחרי החימום שלו הוא בשרי? אפשרי לשתות/לאכול אחריו משהו חלבי? score: 5.92
